<a href="https://colab.research.google.com/github/maxi9113/colab_notebook/blob/main/Taller_Clasification_MLP_Algorith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En el procesamiento de bebidas, la aplicación de un determinado conservante es
realizada en función de la combinación de 4 variables reales, definidas por x1 (cantidad
de agua), x2 (grado de acidez), x3 (temperatura) y x4 (tensión superficial). Se sabe, que
solo existen tres tipos de conservantes que pueden ser aplicados, los cuales son
categorizados como A, B y C. A partir de estas variables, se realizan pruebas de
laboratorio para determinar que tipo de conservante debe ser aplicado en determinada
bebida.

De 148 pruebas experimentales, un equipo de expertos decide aplicar una red MLP
como clasificador de patrones, con el fin de que esta identifique que conservante será
aplicado en un determinado lote de bebida. Por las características de la línea de
producción, se utilizará una red con tres salidas


Arquitectura de la red


*   4 Entradas
*   1 capa escodida 15 Neuronas
*   3 Salidas









In [28]:
import pandas as pd

# Leer el archivo CSV en un DataFrame de pandas, usando ';' como separador
X_Data = pd.read_csv('/content/MLP_Clasification.csv', sep=';')

# Eliminar la columna 'Amostra'
X_Data = X_Data.drop('Amostra', axis=1)

# Mostrar las primeras filas del DataFrame para verificar la carga
display(X_Data.head())

,x1,x2,x3,x4,d1,d2,d3
0,"0,3841","0,2021","0,0000","0,2438",1,0,0
1,"0,1765","0,1613","0,3401","0,0843",1,0,0
2,"0,3170","0,5786","0,3387","0,4192",0,1,0
3,"0,2467","0,0337","0,2699","0,3454",1,0,0
4,"0,6102","0,8192","0,4679","0,4762",0,1,0


In [37]:
import numpy as np

# Separar las columnas de entrada (x1 a x4) y salida (d1 a d3)

x_train = X_Data[['x1', 'x2', 'x3', 'x4']].replace(',', '.', regex=True).astype(float).values
desired = X_Data[['d1', 'd2', 'd3']].values

# Mostrar las formas de los arrays para verificar
print("Shape of x_train:", x_train.shape)
print("Shape of desired:", desired.shape)

Shape of x_train: (130, 4)
Shape of desired: (130, 3)


In [44]:
import numpy as np

# 1. Definir la función de activación y su derivada
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 3. Función para inicializar y entrenar el MLP con arquitectura configurable
def train_mlp_fixed_bias(X, y, hidden_layer_sizes, epochs=100000, learning_rate=0.1, precision_rate=1e-6,  seed=1):
    np.random.seed(seed)

    input_neurons = X.shape[1]
    output_neurons = y.shape[1]

    weights = []
    biases = [] # Esta lista seguirá almacenando los bias, pero serán fijos en 1

    layer_sizes = [input_neurons] + hidden_layer_sizes + [output_neurons]

    print(f"Arquitectura de la red: {layer_sizes[0]} (entrada) -> {' -> '.join(map(str, hidden_layer_sizes))} (ocultas) -> {layer_sizes[-1]} (salida)")

    # Inicializar pesos y sesgos para todas las capas
    for i in range(len(layer_sizes) - 1):
        w = np.random.uniform(low=-1, high=1, size=(layer_sizes[i], layer_sizes[i+1]))
        # MODIFICACIÓN: Los bias ahora se inicializan a 1 y no se actualizarán
        b = np.ones((1, layer_sizes[i+1])) # <-- Aquí se fijan los bias a 1
        weights.append(w)
        biases.append(b) # Aunque no se actualice, lo mantenemos en la estructura para el forward pass



    # 4. Entrenamiento del MLP
    for epoch in range(epochs):
        # Forward Propagation
        layer_outputs = [X]

        for i in range(len(weights)):
            current_input = layer_outputs[-1]
            layer_input_weighted = np.dot(current_input, weights[i]) + biases[i] # Se usa el bias fijo
            layer_output = sigmoid(layer_input_weighted)
            layer_outputs.append(layer_output)

        predicted_output = layer_outputs[-1]

        # Backpropagation
        errors = [None] * len(weights)
        deltas = [None] * len(weights)

        error = y - predicted_output
        errors[-1] = error
        deltas[-1] = error * sigmoid_derivative(predicted_output)

        for i in range(len(weights) - 2, -1, -1):
            error_current_layer = deltas[i+1].dot(weights[i+1].T)
            errors[i] = error_current_layer
            deltas[i] = error_current_layer * sigmoid_derivative(layer_outputs[i+1])

        # Actualizar solo los pesos, los bias permanecen fijos en 1
        for i in range(len(weights)):
            weights[i] += layer_outputs[i].T.dot(deltas[i]) * learning_rate
            # biases[i] += np.sum(deltas[i], axis=0, keepdims=True) * learning_rate # <--- Línea comentada, los bias NO se actualizan

        # Imprimir el error
        loss = np.mean(np.abs(error))
        if (epoch + 1) % 10000 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

        if loss <= precision_rate:
            print(f"✅ Entrenamiento detenido en epoch {epoch + 1} con Loss: {loss:.6f}")
            break

    # 5. Realizar predicciones después del entrenamiento
    def predict(input_data):
        current_output = input_data
        for i in range(len(weights)):
            current_input_weighted = np.dot(current_output, weights[i]) + biases[i]
            current_output = sigmoid(current_input_weighted)
        return current_output

    print("\n--- Predicciones Finales ---")
    final_predictions = predict(X)
    print(final_predictions)
    print("\nValores esperados:")
    print(y)

    return weights, biases, predict


# --- Ejemplo de uso ---
hidden_layer_config = [15] # Aquí configuras tus capas ocultas y neuronas por capa

# Llama a la nueva función que tiene los bias fijos
trained_weights, trained_biases, predict_function = train_mlp_fixed_bias(
    x_train, desired,
    hidden_layer_sizes=hidden_layer_config,
    epochs=500000,
    learning_rate=0.1,
    precision_rate = 1e-6
)

Arquitectura de la red: 4 (entrada) -> 15 (ocultas) -> 3 (salida)
Epoch 10000, Loss: 0.0774
Epoch 20000, Loss: 0.0678
Epoch 30000, Loss: 0.0657
Epoch 40000, Loss: 0.0719
Epoch 50000, Loss: 0.0608
Epoch 60000, Loss: 0.0659
Epoch 70000, Loss: 0.0766
Epoch 80000, Loss: 0.0508
Epoch 90000, Loss: 0.0647
Epoch 100000, Loss: 0.0504
Epoch 110000, Loss: 0.0479
Epoch 120000, Loss: 0.0503
Epoch 130000, Loss: 0.0489
Epoch 140000, Loss: 0.0449
Epoch 150000, Loss: 0.0559
Epoch 160000, Loss: 0.0405
Epoch 170000, Loss: 0.0400
Epoch 180000, Loss: 0.0390
Epoch 190000, Loss: 0.0417
Epoch 200000, Loss: 0.0430
Epoch 210000, Loss: 0.0449
Epoch 220000, Loss: 0.0441
Epoch 230000, Loss: 0.0353
Epoch 240000, Loss: 0.0380
Epoch 250000, Loss: 0.0393
Epoch 260000, Loss: 0.0403
Epoch 270000, Loss: 0.0607
Epoch 280000, Loss: 0.0438
Epoch 290000, Loss: 0.0366
Epoch 300000, Loss: 0.0367
Epoch 310000, Loss: 0.0361
Epoch 320000, Loss: 0.0411
Epoch 330000, Loss: 0.0411
Epoch 340000, Loss: 0.0355
Epoch 350000, Loss: 0.036